In [37]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
df = pd.read_csv('AWD-J.txt',sep="\t")

df.sample(5)

,Word,Org,Score,Deviation,Std,PoS
4630,下書き,下書き,1.97,55.41,1.08,名詞
12767,味わえる,味わえる,2.70,40.27,1.24,動詞
14975,概念,概念,3.27,28.57,1.29,名詞
13805,ベーシック,ベーシック,2.87,36.83,1.48,名詞
13174,ねたい,ねたい,2.77,38.90,1.12,形容詞


In [38]:
import math


df["integer_part"] = df["Score"].apply(lambda x: math.modf(x)[1])
df.sample(5)

,Word,Org,Score,Deviation,Std,PoS,integer_part
10060,佐川,佐川,2.43,45.78,1.15,名詞,2.0
483,座布団,座布団,1.37,67.80,0.71,名詞,1.0
12565,お陰,お陰,2.70,40.27,1.29,名詞,2.0
5103,松井,松井,2.00,54.72,1.37,名詞,2.0
11196,crazy,crazy,2.53,43.71,1.20,名詞,2.0


In [39]:
unique_integer_paths = df["integer_part"].unique()
samples_by_integer_part = []

for score in unique_integer_paths:
    samples_by_integer_part.append(df[df["integer_part"] == score].head(3))

samples_by_integer_part

[   Word   Org  Score  Deviation   Std PoS  integer_part
 0   東京都   東京都   1.00      75.37  0.00  名詞           1.0
 1  稲川淳二  稲川淳二   1.00      75.37  0.00  名詞           1.0
 2  椎名林檎  椎名林檎   1.03      74.68  0.18  名詞           1.0,
      Word Org  Score  Deviation   Std PoS  integer_part
 4825    曲   曲    2.0      54.72  0.89  名詞           2.0
 4826   香り  香り    2.0      54.72  1.15  名詞           2.0
 4827   敬遠  敬遠    2.0      54.72  0.97  名詞           2.0,
       Word  Org  Score  Deviation   Std PoS  integer_part
 14334  ﾁﾗｯ  ﾁﾗｯ    3.0      34.08  1.32  名詞           3.0
 14335  うめー  うめー    3.0      34.08  1.57  名詞           3.0
 14336   次元   次元    3.0      34.08  1.41  名詞           3.0,
        Word   Org  Score  Deviation   Std PoS  integer_part
 15219  ホンワカ  ホンワカ   4.13      10.68  1.06  名詞           4.0]

In [40]:
def df_to_prompt_example(df):
    return f"{df["Word"].iloc[0]}: f{df["Score"].iloc[0]}\n"


In [41]:
prompt_example = ""
for df in samples_by_integer_part:
    prompt_example += df_to_prompt_example(df)
prompt = f"""
あなたは言葉の抽象度を評価するアシスタントです。
ユーザが提供したことの抽象度を判断してください。
以下を参考にしてください。
{prompt_example}

かならず以下のフォーマットに従ってください。
word: score
"""

prompt

'\nあなたは言葉の抽象度を評価するアシスタントです。\nユーザが提供したことの抽象度を判断してください。\n以下を参考にしてください。\n東京都: f1.0\n曲: f2.0\nﾁﾗｯ: f3.0\nホンワカ: f4.13\n\n\nかならず以下のフォーマットに従ってください。\nword: score\n'

In [42]:
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content":"積極的なコミュニケーション,職位の壁を超えたコミュにケーション"}])

response.choices[0].message.content



'積極的なコミュニケーション: f2.5\n職位の壁を超えたコミュニケーション: f2.7'